In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import pyodbc
import urllib
from sqlalchemy import create_engine, text

In [ ]:
# % matplotlib inline
pd.set_option('display.max_columns', 500)

# Load Data

In [6]:
# database = 'LMSMaster'

# # server = '192.168.1.15'
# server = 'BIGDATA'
# username = 'AWang'
# password = '9Kha8H54FyapCq9w'
# # username = 'Cchang'
# # password = 'summerjob2017'
# # params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
# cnxn = engine.connect()


server = '192.168.1.15'
username = 'AWang'
password = '9Kha8H54FyapCq9w'
database = 'LF_LMSMaster'
params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) # ODBC Driver 17 for ODBC Driver 17 for  
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
cnxn = engine.connect()

query1 = f"""  --SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED 

-- new loans originated after 2019-12-01
SELECT L.LoanID, A.PortfolioID, A.Application_ID, SA.ScoredPortfolioID, SA.ScoredAppID, 
C.Frequency, L.OriginatedAmount, L.OriginationDate, SA.ScoredApplicationDate, L.LoanStatus, 
L.NumofInstalments AS MaxInst, FLOOR(DATEDIFF(DAY, C.DOB, A.ApplicationDate)/(365.25)) AS Age,
CASE WHEN EW.EmpName IS NOT NULL THEN 1 ELSE 0 END AS Benefits
INTO #loan_0
FROM Loans L
INNER JOIN Application A ON L.PortFolioID = A.PortFolioID AND L.ApplicationID = A.Application_ID
LEFT JOIN QlikDB..ScoredApplications SA ON L.PortFolioID = SA.PortfolioID AND L.ApplicationID = SA.Application_ID
LEFT JOIN QlikDB..CustomerApplicationData C ON L.PortFolioID = C.PortfolioID AND L.ApplicationID = C.ApplicationID
LEFT JOIN CustomerReports..EmpWhitelist EW ON C.EmpName = EW.EmpName
WHERE A.ApplicationSteps NOT LIKE '%R%' 
AND A.LPCampaign NOT IN ('RTRNGD', 'WEBGD')
AND L.LoanStatus NOT IN ('V','W','G','K')
AND L.OriginationDate BETWEEN '2022-01-01' AND GETDATE()
AND L.OriginatedAmount > 0 AND L.OriginatedAmount <= 1250

-- append default score
SELECT * INTO #loan_1
FROM (
	SELECT T.*, Sresult AS DefScore,
	ROW_NUMBER() OVER (PARTITION BY T.ScoredPortfolioID, T.ScoredAppID ORDER BY S.CreateTime DESC) AS rownum
	FROM #loan_0 T 
	LEFT JOIN LMS_Logs..ScoringRResult_Archive S ON T.ScoredPortfolioID = S.PortfolioID 
		AND T.ScoredAppID = S.ApplicationID AND S.ModelID = 82 AND Sresult IS NOT NULL
) A
WHERE rownum = 1 

ALTER TABLE #loan_1 DROP COLUMN rownum

-- append epo score
SELECT * INTO #loan
FROM (
	SELECT T.*, Sresult AS EPOScore,
	ROW_NUMBER() OVER (PARTITION BY T.ScoredPortfolioID, T.ScoredAppID ORDER BY S.CreateTime DESC) AS rownum
	FROM #loan_1 T 
	LEFT JOIN LMS_Logs..ScoringRResult_Archive S ON T.ScoredPortfolioID = S.PortfolioID 
		AND T.ScoredAppID = S.ApplicationID AND S.ModelID = 94 AND Sresult IS NOT NULL
) A
WHERE rownum = 1 

-- get input variables for payin projection
SELECT L.LoanID, L.LoanStatus, L.OriginatedAmount,
MAX(CASE WHEN I.iPaymentMode = 144 AND DATEDIFF(DAY, I.DueDate, GETDATE()) > 7 AND Status IN (111, 779) 
	THEN I.InstallmentNumber ELSE NULL END) AS MaxInstPaid,
MIN(CASE WHEN I.iPaymentMode = 144 AND DATEDIFF(DAY, I.DueDate, GETDATE()) > 7 AND Status NOT IN (111, 779) 
	THEN I.InstallmentNumber ELSE NULL END) AS MinInstNotPaid,
MAX(CASE WHEN I.iPaymentMode = 144 AND DATEDIFF(DAY, I.DueDate, GETDATE()) > 7 AND I.Status NOT IN (111, 779, 115) 
	THEN 1 ELSE 0 END) AS Defaulted,
SUM(CASE WHEN I.iPaymentMode <> 144 AND DATEDIFF(DAY, P.TransactionDate, GETDATE()) > 7 AND P.PaymentType <> '9' AND P.PaymentStatus <> 'F' 
	THEN PaymentAmount ELSE 0 END) AS ArrPayin,
MAX(CASE WHEN I.iPaymentMode = 144 AND Status IN (111, 779) 
	THEN I.InstallmentNumber ELSE NULL END) AS MaxInstPaidToDate,
MAX(CASE WHEN I.iPaymentMode = 144 AND I.Status NOT IN (111, 779, 115) 
	THEN 1 ELSE 0 END) AS DefaultedToDate,
SUM(CASE WHEN P.PaymentType <> '9' AND P.PaymentStatus <> 'F' THEN PaymentAmount ELSE 0 END) AS PayinToDate,
SUM(CASE WHEN P.PaymentType = '9' AND P.PaymentStatus = 'F' THEN PaymentAmount ELSE 0 END) AS RefundToDate,
SUM(CASE WHEN I.iPaymentMode <> 144 AND P.PaymentType <> '9' AND P.PaymentStatus <> 'F' 
	THEN PaymentAmount ELSE 0 END) AS ArrPayinToDate,
SUM(CASE WHEN I.iPaymentMode <> 144 AND P.PaymentType = '9' AND P.PaymentStatus = 'F' 
	THEN PaymentAmount ELSE 0 END) AS ArrRefundToDate,
MAX(CASE WHEN P.PaymentType <> '9' AND P.PaymentStatus <> 'F' 
	THEN TransactionDate ELSE NULL END) AS DateOfLastPayment
INTO #min_max_inst
FROM #loan L 
INNER JOIN Installments I ON L.LoanID = I.LoanID 
LEFT JOIN Payment P ON I.InstallmentID = P.InstallmentID 
	AND P.PaymentType IN ('I','S','Z','A','X','Q','~','3','9','G','K') 
	AND P.PaymentStatus IN ('D', 'F')
	AND P.PaymentMode NOT IN ('V','P','T','H') 
	AND P.TransactionDate IS NOT NULL
GROUP BY L.LoanID, L.LoanStatus, L.OriginatedAmount

SELECT LoanID, OriginatedAmount, LoanStatus, DateOfLastPayment, Defaulted, MaxInstPaid, MinInstNotPaid, 
PayinToDate - RefundToDate AS PayinToDate, ArrPayinToDate - ArrRefundToDate AS ArrPayinToDate,
CASE WHEN LoanStatus IN ('D', 'P') AND MaxInstPaid >= 1 AND MaxInstPaid = MaxInstPaidToDate 
	AND DefaultedToDate = 0 AND ArrPayinToDate = 0 THEN MaxInstPaid ELSE NULL END AS InstPaidOff,
CASE WHEN MaxInstPaid >= 1 AND Defaulted = 0 AND ArrPayin = 0 
	AND (MinInstNotPaid IS NULL OR (MinInstNotPaid IS NOT NULL AND LoanStatus IN ('N', 'D', 'P')))
THEN MaxInstPaid ELSE NULL END AS MaxInstPaidCont_
INTO #inst_paid
FROM #min_max_inst

SELECT LoanID, Defaulted, LoanStatus, DateOfLastPayment, PayinToDate, InstPaidOff, 
CASE WHEN InstPaidOff IS NULL THEN MaxInstPaidCont_ ELSE NULL END AS MaxInstPaidCont,
CASE WHEN InstPaidOff IS NULL AND MaxInstPaidCont_ IS NULL THEN MinInstNotPaid ELSE NULL END AS InstDefaulted,
CASE WHEN InstPaidOff IS NULL AND MaxInstPaidCont_ IS NULL THEN ArrPayinToDate ELSE NULL END AS ArrPayin
INTO #inst
FROM #inst_paid

-- variables for defaulted accounts
SELECT A.LoanID,
MAX(CASE WHEN A.InstDefaulted = I.InstallmentNumber THEN I.PrincipalBalance ELSE NULL END) AS PrincipalBalance,
MIN(CASE WHEN A.InstDefaulted = I.InstallmentNumber THEN I.DueDate ELSE NULL END) AS DateOfDefault,
SUM(CASE WHEN I.InstallmentNumber <= A.InstDefaulted THEN AmountPaid ELSE 0 END) AS InstPayinPreDefault,
SUM(CASE WHEN I.InstallmentNumber > A.InstDefaulted THEN AmountPaid ELSE 0 END) AS InstPayinPostDefault
INTO #defaulted
FROM #inst A INNER JOIN Installments I ON A.LoanID = I.LoanID
WHERE A.InstDefaulted IS NOT NULL AND I.iPaymentMode = 144
GROUP BY A.LoanID

-- final output
SELECT L.LoanID, L.Application_ID, L.PortfolioID, L.ScoredPortfolioID, L.ScoredAppID, L.Benefits,
DefScore, EPOScore, CASE WHEN L.Frequency = 'S' THEN 'B' ELSE L.Frequency END AS Frequency, 
L.OriginatedAmount, L.OriginationDate, L.ScoredApplicationDate, L.LoanStatus, L.Age, 
L.MaxInst, I.InstDefaulted, I.InstPaidOff, I.MaxInstPaidCont, I.PayinToDate, D.PrincipalBalance, 
D.InstPayinPreDefault, D.InstPayinPostDefault, I.ArrPayin, D.DateOfDefault, I.DateOfLastPayment, 
GETDATE() AS UpdatedDate
INTO #final
FROM #loan L
INNER JOIN #inst I ON L.LoanID = I.LoanID
LEFT JOIN #defaulted D ON L.LoanID = D.LoanID
"""

query2 = "SELECT * FROM #final"
query3 = """
SELECT I.LoanID, InstRate, InstallmentNumber, OriginalFinCharge, OriginalPrincipal, OriginalDueAmount, DueDate 
FROM #final L INNER JOIN Installments I ON L.LoanID = I.LoanID
WHERE (InstallmentNumber <= MaxInst OR InstallmentNumber <= L.InstPaidOff OR InstallmentNumber <= L.InstDefaulted)
AND iPaymentMode = 144
"""
   
cnxn.execute(text(query1))
df_raw = pd.read_sql(text(query2), cnxn)
df_raw_inst = pd.read_sql(text(query3), cnxn)
# df_raw = pd.read_sql(query2, cnxn)
# df_raw_inst = pd.read_sql(query3, cnxn)

DBAPIError: (pyodbc.Error) ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [4]:
df_raw.to_csv('payin_prejection.csv')
df_raw_inst.to_csv('payin_projection_installment.csv')

In [5]:
# df_raw = pd.read_csv('payin_prejection.csv')
# df_raw_inst = pd.read_csv('payin_projection_installment.csv')

In [6]:
# input data for parametric estimation
# df_raw = pd.read_csv('../Data/Raw/payin_projection.csv')
# df_raw_inst = pd.read_csv('../Data/Raw/payin_projection_installment.csv')

# input data for nonparametric estimation
try:
    df_np = pd.read_csv('nonparam_estimation.csv')
except:
    df_np = pd.read_csv('../Data/Raw/nonparam_estimation.csv')

try:
    df_np_inst = pd.read_csv('nonparam_estimation_installment.csv')
except:
    df_np_inst = pd.read_csv('../Data/Raw/nonparam_estimation_installment.csv')

# retro early payoff (epo) band
epo_band_retro = pd.read_csv('epo_band.csv')

In [7]:
# df_raw
import os
cwd = os.getcwd()
cwd, [f for f in os.listdir('.') if os.path.isfile(f)]

('/mnt/batch/tasks/shared/LS_root/mounts/clusters/anson-nbc1/code/Users/andrew/Payin Projection',
 ['.amlignore',
  '.amlignore.amltmp',
  'epo_band.csv',
  'nonparam_estimation.csv',
  'nonparam_estimation_installment.csv',
  'payin_prejection.csv',
  'payin_prejection_installment.csv',
  'payin_projection_03.29.21-v2.ipynb',
  'payin_projection_03.29.21-v2.ipynb.amltmp',
  'payin_projection_03.29.21.ipynb.amltmp',
  'payin_projection_03.29.21_v1.ipynb',
  'payin_projection_03.29.21_v1.ipynb.amltmp',
  'payin_projection_03.29.21_v2.ipynb.amltmp',
  'payin_projection_03.29.21_v3_testbed.ipynb',
  'payin_projection_03.29.21_v3_testbed.ipynb.amltmp',
  'payin_projection_installment.csv',
  'sample.csv'])

In [8]:
# fill missing epo band by retro epo band
df_raw = df_raw.merge(epo_band_retro, how='left', on=['PortfolioID', 'Application_ID'])
df_raw['EPOScore'] = df_raw['EPOScore'].fillna(df_raw['epf_score'])
df_raw.drop(columns=['epf_score', 'epf_Band'], inplace=True)

In [9]:
df_raw

,LoanID,Application_ID,PortfolioID,ScoredPortfolioID,ScoredAppID,Benefits,DefScore,EPOScore,Frequency,OriginatedAmount,OriginationDate,ScoredApplicationDate,LoanStatus,Age,MaxInst,InstDefaulted,InstPaidOff,MaxInstPaidCont,PayinToDate,PrincipalBalance,InstPayinPreDefault,InstPayinPostDefault,ArrPayin,DateOfDefault,DateOfLastPayment,UpdatedDate
0,I1433384-0,61399619,1,5.0,95683901.0,1,431.0,306.0,M,900.0,2022-09-16 11:31:39.203,2022-09-16 10:28:29.790,C,67.0,18,3.0,NaN,NaN,720.00,900.0,745.0,0.0,0.0,2022-11-30,2022-10-28 14:26:19.417,2024-07-22 10:28:39.630
1,I1453174-0,81748269,6,5.0,98085038.0,0,445.0,266.0,W,800.0,2022-10-14 11:44:25.953,2022-10-14 10:54:18.993,C,52.0,18,9.0,NaN,NaN,1504.00,480.0,1504.0,0.0,0.0,2022-12-16,2022-12-08 17:23:11.407,2024-07-22 10:28:39.630
2,I1457913-0,64729610,1,5.0,98564041.0,1,410.0,330.0,M,900.0,2022-10-20 12:17:29.583,2022-10-20 11:36:42.340,C,50.0,18,3.0,NaN,NaN,720.00,900.0,770.0,0.0,0.0,2023-01-03,2022-12-08 16:27:02.400,2024-07-22 10:28:39.630
3,I1467711-0,99448418,5,5.0,99447757.0,0,356.0,708.0,B,900.0,2022-11-01 12:01:11.460,2022-11-01 10:48:44.150,D,42.0,18,NaN,1.0,NaN,1051.88,NaN,NaN,NaN,NaN,NaT,2022-11-09 16:34:03.647,2024-07-22 10:28:39.630
4,I1493405-0,101561933,5,5.0,101561110.0,0,487.0,290.0,B,800.0,2022-11-29 09:20:29.043,2022-11-29 09:06:58.477,C,51.0,18,1.0,NaN,NaN,250.00,800.0,275.0,0.0,0.0,2022-12-09,2023-01-20 09:02:58.613,2024-07-22 10:28:39.630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79885,I1721372-0,79010804,7,7.0,79010796.0,0,438.0,629.0,B,900.0,2023-07-07 21:15:39.980,2023-07-07 21:08:45.600,D,38.0,18,NaN,8.0,NaN,2944.22,NaN,NaN,NaN,NaN,NaT,2023-11-10 07:45:57.950,2024-07-22 10:28:39.630
79886,I1905523-0,99445512,6,7.0,84832286.0,0,484.0,NaN,B,600.0,2023-12-21 12:23:03.580,2023-11-20 13:46:18.407,C,49.0,16,2.0,NaN,NaN,780.00,600.0,180.0,600.0,0.0,2024-01-30,2024-03-27 10:33:28.833,2024-07-22 10:28:39.630
79887,I1924422-0,86215786,1,7.0,86141162.0,0,481.0,NaN,B,600.0,2024-01-12 17:32:01.973,2024-01-08 12:06:46.673,C,52.0,16,2.0,NaN,NaN,180.00,600.0,205.0,0.0,0.0,2024-02-02,2024-01-23 14:03:16.763,2024-07-22 10:28:39.630
79888,I1393615-0,78221040,6,6.0,78220941.0,0,455.0,352.0,B,800.0,2022-07-15 12:43:22.727,2022-07-15 08:42:50.763,D,47.0,18,NaN,13.0,NaN,3196.00,NaN,NaN,NaN,NaN,NaT,2023-01-27 06:47:32.020,2024-07-22 10:28:39.630


# Data validation

### Convert strings to datetime object

In [10]:
date_cols = [col for col in df_raw.columns if 'Date' in col and 'ToDate' not in col]
df_raw.loc[:, date_cols] = df_raw.loc[:, date_cols].apply(pd.to_datetime, errors='coerce')

date_cols = [col for col in df_raw_inst.columns if 'Date' in col and 'ToDate' not in col]
df_raw_inst.loc[:, date_cols] = df_raw_inst.loc[:, date_cols].apply(pd.to_datetime, errors='coerce')

### Define MyLoanStatus (live production version)

In [11]:
df_raw.loc[:, 'MyLoanStatus'] = df_raw['LoanStatus'].map({'D': 'paid_off', 'P': 'paid_off', 
                                                          'A': 'settled', 'L': 'debt_con', 
                                                          'B': 'bankruptcy', 'C': 'charge_off'})

mask_1 = ((df_raw['UpdatedDate'] - df_raw['DateOfDefault']).dt.days >= 120) \
& (df_raw['InstPayinPostDefault'] == 0)

mask_2 = (df_raw['InstPayinPostDefault'] > 0) \
& ((df_raw['UpdatedDate'] - df_raw['DateOfLastPayment']).dt.days >= 60)

df_raw.loc[(mask_1 | mask_2) & df_raw['MyLoanStatus'].isnull(), 'MyLoanStatus'] = 'charge_off'

mask = df_raw['MyLoanStatus'].isnull()

df_raw.loc[mask, 'MyLoanStatus'] = np.where(df_raw.loc[mask, 'InstDefaulted'].notnull(), 
                                            'defaulted', 
                                            'current')

### Flag invalid values
Error: we cannot predict the payin of this loan due to invalid values in the required fields

Warning: 
> missing gen3 default score or early-payoff score 

> pay frequency changed after loan origination

In [12]:
# configuration
valid_loan_status = ['D', 'P', 'A', 'L', 'B', 'C', 'N', 'R']
rate_freq_map = {15: 'W', 20: 'W', 30: 'B', 50: 'M'}
min_loan_amt = 200
min_age = 24
max_age = 100
min_defscore = 0
max_defscore = 1000
min_eposcore = 0
max_eposcore = 1000

In [13]:
df_raw.loc[df_raw['EPOScore'].notnull(),'EPOScore'].mean()

479.3827697684578

In [14]:
df_raw.loc[(df_raw['EPOScore'].notnull()),'EPOScore'] = [int(i) for i in df_raw.loc[(df_raw['EPOScore'].notnull()),'EPOScore'].values]
df_raw.loc[(df_raw['DefScore'].notnull()),'DefScore'] = [int(i) for i in df_raw.loc[(df_raw['DefScore'].notnull()),'DefScore'].values]

In [15]:
df_raw['EPOScore'] = df_raw['EPOScore'].fillna(df_raw.loc[df_raw['EPOScore'].notnull(),'EPOScore'].mean())
df_raw['DefScore'] = df_raw['DefScore'].fillna(df_raw.loc[df_raw['DefScore'].notnull(),'DefScore'].mean())
df_raw.loc[df_raw['EPOScore']== None, 'EPOScore'] = df_raw.loc[df_raw['EPOScore'].notnull(),'EPOScore'].mean()
df_raw.loc[df_raw['DefScore']== None, 'DefScore'] = df_raw.loc[df_raw['DefScore'].notnull(),'DefScore'].mean()

In [16]:
# for parametric estimation
# flag loans with invalid values in the required fields
df_raw.loc[(df_raw['PayinToDate'] == 0) \
           & ( (df_raw['MaxInstPaidCont'] >= 1) | (df_raw['InstPaidOff'] >= 1) ), 
           'Error'] = 'installment_table_error'

df_raw.loc[df_raw['MaxInstPaidCont'] > df_raw['MaxInst'], 'Error'] = 'loan_table_error'

df_raw.loc[~((df_raw['Age'] >= min_age) & (df_raw['Age'] <= max_age)), 'Error'] = 'invalid_age'

df_raw.loc[~df_raw['LoanStatus'].isin(valid_loan_status), 'Error'] = 'invalid_loan_status'

df_raw.loc[~(df_raw['OriginatedAmount'] >= min_loan_amt), 'Error'] = 'loan_amount_too_low'

# get the most common interest rate for each loan from installment-level data
most_common_inst_rate = df_raw_inst.groupby(['LoanID'])['InstRate']\
.apply(lambda s: s.value_counts().index[0]).to_frame('MostCommIntRate').reset_index()
df_raw = df_raw.merge(most_common_inst_rate, how='left', on='LoanID')
df_raw.loc[df_raw['MostCommIntRate'].isnull(), 'Error'] = 'missing_interest_rate'

# flag loans of which the interest rate doesn't match pay frequency
mask = False
for rate, freq in rate_freq_map.items():
    mask = mask | ((df_raw['Frequency'] == freq) & (df_raw['MostCommIntRate'] == rate))
df_raw.loc[~mask, 'Warning'] = 'frequency_interest_mismatch'

# modify the Frequency to make it align with MostCommIntRate 
mask = df_raw['MostCommIntRate'].notnull()
df_raw.loc[mask, 'Frequency'] = df_raw.loc[mask, 'MostCommIntRate'].map(rate_freq_map)

# warnings for missing gen3 scores
df_raw.loc[~((df_raw['EPOScore'] >= min_eposcore) & (df_raw['EPOScore'] <= max_eposcore)), 
           'Warning'] = 'invalid_epo_score'

df_raw.loc[~((df_raw['DefScore'] >= min_defscore) & (df_raw['DefScore'] <= max_defscore)), 
           'Warning'] = 'invalid_def_score'

In [17]:
# for non-parametric estimation (ignore borrower's characteristics)
# flag loans with invalid values in the required fields
df_np.loc[(df_np['PayinToDate'] == 0) & ((df_np['MaxInstPaidCont'] >= 1) | (df_np['InstPaidOff'] >= 1)), 
          'Error'] = 'installment_table_error'

df_np.loc[~df_np['LoanStatus'].isin(valid_loan_status), 'Error'] = 'invalid_loan_status'

df_np.loc[~(df_np['OriginatedAmount'] >= min_loan_amt), 'Error'] = 'loan_amount_too_low'

# get the most common interest rate for each loan from installment-level data
most_common_inst_rate = df_np_inst.groupby(['LoanID'])['InstRate'].apply(lambda s: s.value_counts().index[0])\
.to_frame('MostCommIntRate').reset_index()
df_np = df_np.merge(most_common_inst_rate, how='left')
df_np.loc[df_np['MostCommIntRate'].isnull(), 'Error'] = 'missing_interest_rate'

# flag loans of which the interest rate doesn't match pay frequency
mask = False
for rate, freq in rate_freq_map.items():
    mask = mask | ((df_np['Frequency'] == freq) & (df_np['MostCommIntRate'] == rate))
df_np.loc[~mask, 'Warning'] = 'frequency_interest_mismatch'

# modify the Frequency to make it align with MostCommIntRate 
mask = df_np['MostCommIntRate'].notnull()
df_np.loc[mask, 'Frequency'] = df_np.loc[mask, 'MostCommIntRate'].map(rate_freq_map)

### Number of loans with invalid values
For parametric estimation

In [18]:
df_raw['Error'].value_counts()

invalid_age    3
Name: Error, dtype: int64

In [19]:
df_raw['Warning'].value_counts()

frequency_interest_mismatch    2301
Name: Warning, dtype: int64

For non-parametric estimation

In [20]:
df_np['Error'].value_counts()

missing_interest_rate    12
Name: Error, dtype: int64

In [21]:
df_np['Warning'].value_counts()

frequency_interest_mismatch    168
Name: Warning, dtype: int64

### Seperate out loans with error flag

In [22]:
# for parametric estimation
mask = df_raw['Error'].notnull()
df_error = df_raw[mask].copy()
df = df_raw[~mask].copy()
df_inst = df_raw_inst[~df_raw_inst['LoanID'].isin(df_error['LoanID'])].copy()

In [23]:
# for non-parametric estimation
df_np = df_np[df_np['Error'].isnull()]

# Preprocess Data

In [24]:
# scale scores
score_cols = ['DefScore', 'EPOScore']
for score in score_cols:
    df.loc[:, score] = df.loc[:, score] / 1000

In [25]:
df['AgeGroup'] = 0

In [26]:
# cut age into groups 
d_age_bins = {'W': [40, 60, 101], 
              'M': [60, 65, 70, 80, 101],
              'B': [35, 40, 45, 55, 101]}
d_age_labels = {'W': ['40_59', '60_100'], 
                'M': ['60_64', '65_69', '70_79', '80_100'], 
                'B': ['35_39', '40_44', '45_54', '55_100']}

for freq in ['W', 'B', 'M']:
    mask = (df['Frequency'] == freq)
    df.loc[mask, 'AgeGroup'] = pd.cut(df.loc[mask, 'Age'], bins=d_age_bins[freq],
                                    labels=d_age_labels[freq], right=False, include_lowest=True).values

# Define Calibration Factors

In [27]:
# create an empty Dataframe to store the calibration factors by frequency by installment
df_cf = pd.DataFrame({'Frequency': ['W']*19 + ['B']*19 + ['M']*19, 
                      'InstallmentNumber': list(range(1, 20)) * 3})
df_cf

,Frequency,InstallmentNumber
0,W,1
1,W,2
2,W,3
3,W,4
4,W,5
5,W,6
6,W,7
7,W,8
8,W,9
9,W,10


### Recovery Rate (RR)

In [28]:
df_def = df[(df['UpdatedDate'] - df['DateOfDefault']).dt.days >= 120].copy()

# exclude defaults occured during COVID outbreak
df_def = df_def.merge(df_inst[['LoanID', 'InstallmentNumber', 'DueDate']], how='left', 
                      left_on=['LoanID', 'InstDefaulted'], right_on=['LoanID', 'InstallmentNumber'])

df_def = df_def[(df_def['DueDate'] < datetime(2020, 2, 1)) | (df_def['DueDate'] > datetime(2020, 5, 11))]

df_rr = df_def.groupby(['Frequency', 'InstDefaulted']).agg(
    NumOfLoans=('LoanID', 'count'),
    PrincipalBalance=('PrincipalBalance', 'sum'),
    InstPayinPostDefault=('InstPayinPostDefault', 'sum'),
    ArrPayin=('ArrPayin', 'sum')
)

df_rr = df_rr[df_rr['NumOfLoans'] >= 20]

df_rr['_RR'] = (df_rr['InstPayinPostDefault'] + df_rr['ArrPayin']) / df_rr['PrincipalBalance']

# df_rr['_RR'].to_clipboard()

df_cf = df_cf.merge(df_rr[['_RR']], left_on=['Frequency', 'InstallmentNumber'], 
                    right_index=True, how='left')

df_cf['RR'] = df_cf.groupby('Frequency')['_RR'].apply(lambda s: s.fillna(s.mean())) * 0.8

### Calibration factor for the pre-default installment payments (CFD)

In [29]:
# theoritical payin pre-default
df_inst_dtp = df_inst.merge(df.loc[df['InstDefaulted'].notnull(), 
                                   ['LoanID', 'Frequency', 'InstDefaulted', 'InstPayinPreDefault']])

df_inst_dtp = df_inst_dtp[df_inst_dtp['InstallmentNumber'] < df_inst_dtp['InstDefaulted']]

df_dtp = df_inst_dtp.groupby('LoanID').agg(
    Frequency=('Frequency', 'first'),
    InstDefaulted=('InstDefaulted', 'first'),
    InstPayinPreDefault=('InstPayinPreDefault', 'first'),
    TPD=('OriginalDueAmount', 'sum')
)

df_dcf = df_dtp.groupby(['Frequency', 'InstDefaulted']).agg(
    NumOfLoans=('TPD', 'count'),
    TPD=('TPD', 'sum'),
    InstPayinPreDefault=('InstPayinPreDefault', 'sum')
)

df_dcf = df_dcf[df_dcf['NumOfLoans'] >= 20]

df_dcf['_CFD'] = df_dcf['InstPayinPreDefault'] / df_dcf['TPD']

df_cf = df_cf.merge(df_dcf[['_CFD']], left_on=['Frequency', 'InstallmentNumber'], 
                    right_index=True, how='left')

df_cf['CFD'] = df_cf.groupby('Frequency')['_CFD'].apply(lambda s: s.fillna(1))

df_cf.loc[df_cf['InstallmentNumber'] == 1, 'CFD'] = 0

df = df.merge(df_dtp[['TPD']], how='left', left_on='LoanID', right_index=True)
df.loc[df['InstDefaulted'] == 1, 'TPD'] = 0

### Calibration factor for the payin of early-payoff accounts (CFE)

In [30]:
# theoritical payin of early-payoff accounts
df_inst_etp = df_inst.merge(df.loc[df['InstPaidOff'].notnull() & (df['MyLoanStatus'] == 'paid_off'), 
                                   ['LoanID', 'Frequency', 'InstPaidOff', 'OriginatedAmount', 'PayinToDate']])

df_inst_etp = df_inst_etp[df_inst_etp['InstallmentNumber'] <= df_inst_etp['InstPaidOff']]

df_etp = df_inst_etp.groupby('LoanID').agg(
    Frequency=('Frequency', 'first'),
    InstPaidOff=('InstPaidOff', 'first'),
    PayinToDate=('PayinToDate', 'first'),
    OriginatedAmount=('OriginatedAmount', 'first'),
    OriginalDueAmount=('OriginalDueAmount', 'sum'),
    OriginalPrincipal=('OriginalPrincipal', 'sum')
)

df_etp['TPE'] = df_etp['OriginalDueAmount'] + (df_etp['OriginatedAmount'] - df_etp['OriginalPrincipal'])

df_ecf = df_etp.groupby(['Frequency', 'InstPaidOff']).agg(
    NumOfLoans=('OriginatedAmount', 'count'),
    PayinToDate=('PayinToDate', 'sum'),
    TPE=('TPE', 'sum')
)

df_ecf = df_ecf[df_ecf['NumOfLoans'] >= 20]

df_ecf['_CFE'] = df_ecf['PayinToDate'] / df_ecf['TPE']

df_cf = df_cf.merge(df_ecf[['_CFE']], left_on=['Frequency', 'InstallmentNumber'], 
                    right_index=True, how='left')

df_cf['CFE'] = df_cf.groupby('Frequency')['_CFE'].apply(lambda s: s.fillna(s.mean()))

df = df.merge(df_etp[['TPE']], how='left', left_on='LoanID', right_index=True)

In [31]:
df_rr

NumOfLoans  PrincipalBalance  InstPayinPostDefault  \
Frequency InstDefaulted                                                       
B         1.0                  6162        4660680.00             525885.37   
          2.0                  5167        3887216.77             585483.94   
          3.0                  3223        2413954.96             397270.03   
          4.0                  2370        1742124.79             265545.34   
          5.0                  2734        1981336.19             489138.09   
          6.0                  1437         946012.06             200072.19   
          7.0                   949         553769.07             124457.33   
          8.0                   630         322195.77              81226.18   
          9.0                   419         188140.08              46720.60   
          10.0                  281         105920.86              32652.80   
          11.0                  213          72027.40              21641.00   
          12.0                  158          46947.72              15507.40   
          13.0                  122          32473.58               6965.18   
          14.0                   83          18245.86               5856.00   
          15.0                   61          10710.00               2585.00   
          16.0                   49           6423.08               1532.29   
          17.0                   28           2750.00                807.00   
          18.0                   25           1280.00                180.00   
M         1.0                  2409        1723540.00             182317.50   
          2.0                  2414        1730258.38             214698.34   
          3.0                  1495        1047792.06             186209.47   
          4.0                   920         634391.43              95868.00   
          5.0                   794         526781.71             111305.35   
          6.0                   421         255139.61              57932.66   
          7.0                   220         113454.33              41770.17   
          8.0                   140          63482.32              23085.00   
          9.0                   101          40808.21              17475.00   
          10.0                   67          24075.00               6410.00   
          11.0                   37          12045.00               3385.00   
          12.0                   33           9280.00               3797.50   
W         1.0                  1898        1422200.00             159125.50   
          2.0                  1425        1073121.28             215239.30   
          3.0                   981         730380.00             121500.69   
          4.0                   772         573512.79              90321.16   
          5.0                  1082         791494.85             169965.11   
          6.0                   575         383726.96              86408.00   
          7.0                   379         221415.08              43327.00   
          8.0                   259         137092.06              34349.04   
          9.0                   208          92889.85              17693.45   
          10.0                  142          53495.52              12926.44   
          11.0                   96          34756.37              10546.20   
          12.0                   94          28157.43               6445.99   
          13.0                   79          20895.57               4607.00   
          14.0                   42           9985.00               1222.00   
          15.0                   33           5910.00               1839.50   
          16.0                   41           5402.00               1144.80   
          17.0                   24           2680.00                372.00   

                          ArrPayin       _RR  
Frequency InstDefaulted                       
B         1.0            378099.11  0.193960  
          2.

# Prepare Data For Survival Analysis

### Loan-Level Dataset

In [32]:
# EventType = 0: paid & continue 
# EventType = 1: defaulted
# EventType = 2: paid off

df.loc[:, 'EventTime'] \
= df['InstPaidOff'].fillna(df['InstDefaulted']).fillna(df['MaxInstPaidCont'])

df_np.loc[:, 'EventTime'] \
= df_np['InstPaidOff'].fillna(df_np['InstDefaulted']).fillna(df_np['MaxInstPaidCont'])

for k,v in {'MaxInstPaidCont': 0, 'InstDefaulted': 1, 'InstPaidOff': 2}.items():
    df.loc[df[k].notnull(), 'EventType'] = v
    df_np.loc[df_np[k].notnull(), 'EventType'] = v

### Specify training set

In [33]:
# event time is not null
df_train = df[df['EventTime'].notnull()].copy()

### Loan-Installment Dataset

In [34]:
# for parametric estimation
useful_cols = ['LoanID', 'EventTime', 'EventType', 'OriginationDate', 'Frequency', 'MaxInst', 
               'AgeGroup', 'DefScore', 'EPOScore', 'Age', 'Benefits', 'UpdatedDate']

# replicate rows in the DataFrame by column EventTime
df_train_li = df_train.loc[np.repeat(df_train.index.values, df_train['EventTime']), useful_cols].reset_index(drop=True)
df_train_li.loc[:, 'InstallmentNumber'] = df_train_li.groupby('LoanID').cumcount() + 1
df_train_li.loc[:, 'Target'] = np.where(df_train_li['InstallmentNumber'] == df_train_li['EventTime'], 
                                        df_train_li['EventType'], 0)
df_train_li.loc[:, 'Default'] = np.where(df_train_li['Target'] == 1, 1, 0)
df_train_li.loc[:, 'EPO'] = np.where(df_train_li['Target'] == 2, 1, 0)

In [35]:
# append DueDate to each installment
df_train_li = df_train_li.merge(df_inst[['LoanID', 'InstallmentNumber', 'DueDate']], how='left')
df_train_li['DaysSinceDueDate'] = (df_train_li['UpdatedDate'] - df_train_li['DueDate']).dt.days
df_train_li['DaysSinceDueDateBinned'] = pd.cut(df_train_li['DaysSinceDueDate'], 
                                               bins=[0, 30, 60, 90, np.inf], 
                                               labels=['0-30', '31-60', '61-90', '91+'],
                                               include_lowest=True)

In [36]:
# for non-parametric estimation
useful_cols_np = ['LoanID', 'EventTime', 'EventType', 'Frequency'] #, 'MaxInst', 'Age']

# replicate rows in the DataFrame by column EventTime
df_np_li = df_np.loc[np.repeat(df_np.index.values, df_np['EventTime']), useful_cols_np].reset_index(drop=True)
df_np_li.loc[:, 'InstallmentNumber'] = df_np_li.groupby('LoanID').cumcount() + 1
df_np_li.loc[:, 'Target'] = np.where(df_np_li['InstallmentNumber'] == df_np_li['EventTime'], 
                                     df_np_li['EventType'], 0)
df_np_li.loc[:, 'Default'] = np.where(df_np_li['Target'] == 1, 1, 0)
df_np_li.loc[:, 'EPO'] = np.where(df_np_li['Target'] == 2, 1, 0)

# Survival Analysis
### Estimating Sample Hazard & Survival Probabilities (non-parametric)

discrete-time hazard is the conditional probability that default (or early payoff) will accour at installment i, given that it hasn't occured earlier
<br> h(i) = # defaulted at installment i / # at risk at installment i

survival probability describes the chance that a borrower will survive beyond the installment i without experiencing default or early payoff
<br> S(i) = S(i-1)\[1-h(i)\]

In [37]:
# number of samples by Frequency by Installment
df_sample_size = df_train_li.groupby(['Frequency', 'InstallmentNumber']).size()\
.to_frame('NumOfLoans').reset_index()

# Max installment with at least 30 samples
max_valid_inst = df_sample_size.loc[df_sample_size['NumOfLoans'] >= 30, :]\
.groupby('Frequency')['InstallmentNumber'].max()

# only estimate the hazard/survival probability up to the installments with at least 30 samples
df_train_li['MaxValidInst'] = df_train_li['Frequency'].map(max_valid_inst)
df_train_li = df_train_li[df_train_li['InstallmentNumber'] <= df_train_li['MaxValidInst']]

In [38]:
max_valid_inst

Frequency
B    19
M    18
W    19
Name: InstallmentNumber, dtype: int64

Hazard Rate of Gen3 vs. Gen2

In [39]:
def get_survival_probs(data):
    res = data.groupby(['Frequency', 'InstallmentNumber'])\
    .agg({'Default': ['count', 'sum'], 'EPO': 'sum'})
    res.columns = ['AtRisk', 'Default', 'EPO']
    res['DefHazard'] = res['Default'] / res['AtRisk']
    res['EPOHazard'] = res['EPO'] / res['AtRisk']
    res['PCHazard'] = 1 - res['DefHazard'] - res['EPOHazard']
    res['SurvivalProb'] = res['PCHazard'].cumprod()
    res['SurvivalProbPreInst'] = res['SurvivalProb'].shift(1).fillna(1)
    res['DefAbsRisk'] = res['DefHazard'] * res['SurvivalProbPreInst']
    res['EPOAbsRisk'] = res['EPOHazard'] * res['SurvivalProbPreInst']
    res['CumDefRate'] = res['DefAbsRisk'].cumsum()
    res['CumEPORate'] = res['EPOAbsRisk'].cumsum()
    res = res.reset_index()
    return res

In [40]:
p_survival = get_survival_probs(df_train_li)
p_survival['ScoringModel'] = 'Gen3'

np_survival = get_survival_probs(df_np_li)
np_survival['ScoringModel'] = 'Gen2'

res = pd.concat([p_survival, np_survival])

np_survival.loc[:, 'MaxValidInst'] = np_survival['Frequency'].map(max_valid_inst)

In [41]:
# for freq in ['W', 'B', 'M']:
    # res[(res['Frequency'] == freq) & (res['InstallmentNumber'] <= 18)].groupby(['InstallmentNumber', 'ScoringModel'])['DefHazard'].first().unstack().plot(title=f'Default Hazard Probability - {freq}')

In [42]:
for freq in ['W', 'B', 'M']:
    print(res[(res['Frequency'] == freq) & (res['InstallmentNumber'] <= 18)].groupby(['InstallmentNumber', 'ScoringModel'])['DefHazard'].first().unstack())

ScoringModel           Gen2      Gen3
InstallmentNumber                    
1                  0.085714  0.150342
2                  0.088112  0.142449
3                  0.098746  0.117665
4                  0.072824  0.110718
5                  0.108696  0.178552
6                  0.121839  0.116119
7                  0.084699  0.091355
8                  0.073620  0.073935
9                  0.044674  0.066503
10                 0.041199  0.052502
11                 0.032922  0.041579
12                 0.021739  0.043040
13                 0.041475  0.041319
14                 0.050761  0.025353
15                 0.052326  0.022688
16                 0.040268  0.027850
17                 0.038462  0.022318
18                 0.000000  0.021223
ScoringModel           Gen2      Gen3
InstallmentNumber                    
1                  0.153418  0.163173
2                  0.156002  0.176948
3                  0.135885  0.139741
4                  0.124946  0.128438
5           

In [43]:
for freq in ['W', 'B', 'M']:
    print(res[res['Frequency'] == freq].groupby(['InstallmentNumber', 'ScoringModel'])['EPOHazard']\
    .first().unstack())

ScoringModel           Gen2      Gen3
InstallmentNumber                    
1                  0.026087  0.041901
2                  0.019580  0.023195
3                  0.018809  0.023877
4                  0.028419  0.021364
5                  0.031621  0.028307
6                  0.036782  0.035438
7                  0.024590  0.035970
8                  0.033742  0.051801
9                  0.037801  0.038703
10                 0.048689  0.061099
11                 0.020576  0.047519
12                 0.034783  0.069019
13                 0.050691  0.051429
14                 0.076142  0.129693
15                 0.081395  0.071553
16                 0.436242  0.212435
17                 0.192308  0.284120
18                 0.950000  0.791511
19                 0.666667  0.898649
ScoringModel           Gen2      Gen3
InstallmentNumber                    
1                  0.056288  0.071321
2                  0.045486  0.042059
3                  0.041803  0.037585
4           

### Introduce the Predictors into the Loan-Installment Dataset 

In [44]:
# reversed installment number
df_train_li['ReversedInst'] = df_train_li['MaxInst'] - df_train_li['InstallmentNumber']

In [45]:
# create interactions between Installment and DefScore/EPOScore
interactions = ['InstDefScore', 'InstEPOScore']
df_train_li.loc[:, 'InstDefScore'] = df_train_li.loc[:, 'InstallmentNumber'] * df_train_li.loc[:, 'DefScore']
df_train_li.loc[:, 'InstEPOScore'] = df_train_li.loc[:, 'InstallmentNumber'] * df_train_li.loc[:, 'EPOScore']

### Oversample the recent observations

In [46]:
multiplier = np.array([2, 2, 1.5, 1])

In [47]:
original_count = df_train_li.groupby('DaysSinceDueDateBinned').size()
original_dist = original_count / original_count.sum()
print('original distribution by DaysSinceDueDateBinned:\n', original_dist)

original distribution by DaysSinceDueDateBinned:
 DaysSinceDueDateBinned
0-30     0.020608
31-60    0.042268
61-90    0.040386
91+      0.896739
dtype: float64


In [48]:
oversampled_count = (original_count * multiplier).astype(int)
oversampled_dist = oversampled_count / oversampled_count.sum()
print('distribution by DaysSinceDueDateBinned after oversampling:\n', oversampled_dist)

distribution by DaysSinceDueDateBinned after oversampling:
 DaysSinceDueDateBinned
0-30     0.038054
31-60    0.078052
61-90    0.055932
91+      0.827962
dtype: float64


In [49]:
# oversample observations in 0-30, 31-60, 61-90 bins and concatenate the resulting dataframes 
ros_list = []
for k,v in oversampled_count.to_dict().items():
    ros_list.append(df_train_li[df_train_li['DaysSinceDueDateBinned'] == k]\
                    .sample(n=v, replace=True, random_state=1).copy())
df_train_li_ros = pd.concat(ros_list).reset_index(drop=True)

### Fit Multinomial Logistic Model

In [50]:
# create empty dictionaries to store model object and input variables for different products
d_lr = dict()
d_lr_wos = dict() # model without gen3 scores
d_var = dict()
d_var_wos = dict() # model without gen3 scores

In [51]:
def fit_model(data, freq, max_valid_inst=max_valid_inst, interactions=interactions):
    # build model for a specific product
    df_li = data[data['Frequency'] == freq].copy()
    
    # one-hot encoding
    # add a set of dummy predictors to represent installments
    time_ind =[f'D{i}' for i in range(1, max_valid_inst[freq] + 1)]
    df_time_ind = pd.get_dummies(df_li['InstallmentNumber'], prefix='D', prefix_sep='')
    df_li = pd.concat([df_li, df_time_ind], axis = 1)
    
    # add a set of dummy predictors to represent age groups
    df_age_group = pd.get_dummies(df_li['AgeGroup'], prefix='Age', prefix_sep='_')
    df_li = pd.concat([df_li, df_age_group], axis = 1)
    age_groups = list(df_age_group.columns)
    
    # save input variable names
    var_names = time_ind + age_groups + ['DefScore', 'EPOScore', 'ReversedInst'] + interactions
    if freq == 'M':
        var_names.append('Benefits')
    
    # fit the multinomial logistic regression to data
    X = df_li[var_names] 
    y = df_li[['Target']] 
    lr = sm.MNLogit(y, X).fit_regularized(method='l1') 
    print(f'\nFrequency: {freq}', f'AIC: {lr.aic}')
    print(lr.summary()) 
    
    return lr, var_names

In [52]:
# we also train special models that only use age and installment numbers as inputs
# in case some accounts don't have default or EPO scores
def fit_model_wos(data, freq, max_valid_inst=max_valid_inst):
    # build model for a specific product
    df_li = data[data['Frequency'] == freq].copy()
    
    # one-hot encoding
    # add a set of dummy predictors to represent installments
    time_ind =[f'D{i}' for i in range(1, max_valid_inst[freq] + 1)]
    df_time_ind = pd.get_dummies(df_li['InstallmentNumber'], prefix='D', prefix_sep='')
    df_li = pd.concat([df_li, df_time_ind], axis = 1)
    
    # add a set of dummy predictors to represent age groups
    df_age_group = pd.get_dummies(df_li['AgeGroup'], prefix='Age', prefix_sep='_')
    df_li = pd.concat([df_li, df_age_group], axis = 1)
    age_groups = list(df_age_group.columns)
    
    # save input variable names
    var_names = time_ind + age_groups + ['ReversedInst']
    if freq == 'M':
        var_names.append('Benefits')
    
    # fit the multinomial logistic regression to data
    X = df_li[var_names] 
    y = df_li[['Target']] 
    lr = sm.MNLogit(y, X).fit_regularized(method='l1') 
    print(f'\nFrequency: {freq}', f'AIC: {lr.aic}')
    print(lr.summary()) 
    
    return lr, var_names

In [53]:
mask = df_train_li_ros['DefScore'].notnull() & df_train_li_ros['EPOScore'].notnull()

for freq in ['B', 'W', 'M']:
    # transform data and fit model 
    lr, var_names = fit_model(df_train_li_ros[mask], freq)
    lr_wos, var_names_wos = fit_model_wos(df_train_li_ros, freq)

    # save model objects and input variable names 
    d_lr[freq] = lr
    d_lr_wos[freq] = lr_wos
    d_var[freq] = var_names
    d_var_wos[freq] = var_names_wos

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5688510085724973
            Iterations: 475
            Function evaluations: 478
            Gradient evaluations: 475

Frequency: B AIC: 280751.2319731901
                          MNLogit Regression Results                          
Dep. Variable:                 Target   No. Observations:               246672
Model:                        MNLogit   Df Residuals:                   246616
Method:                           MLE   Df Model:                           54
Date:                Mon, 22 Jul 2024   Pseudo R-squ.:                 0.08125
Time:                        14:31:03   Log-Likelihood:            -1.4032e+05
converged:                       True   LL-Null:                   -1.5273e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
    Target=1       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------

https://stats.stackexchange.com/questions/454415/how-to-account-for-the-recency-of-the-observations-in-a-regression-problem

In [54]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler

# # standardization
# scaler = StandardScaler()
# print(scaler.fit(x_freq))
# x_freq_scaled = pd.DataFrame(scaler.transform(x_freq), index=x_freq.index, columns=x_freq.columns)

# # specify sample weight
# k = 0.005
# sample_weight = np.exp(-k * df_li_freq['DaysSinceDueDate'])
# sample_weight.name='SampleWeight'
# df_li_freq = pd.concat([df_li_freq, sample_weight], axis=1)
# df_li_freq.sort_values(by='DaysSinceDueDate')[['DaysSinceDueDate', 'SampleWeight']].set_index('DaysSinceDueDate').plot()

# # fit multinomial logistic regression to data
# lr = LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg', fit_intercept=False)\
# .fit(x_freq_scaled, y_freq.values.ravel(), sample_weight=sample_weight)

# Make Prediction
### For loans in current status

In [55]:
# select accounts neither defaulted nor paid off
df_pred = df[df['MyLoanStatus'] == 'current'].copy()

#### Installment-level dataset

In [56]:
# for each loan duplicate the rows n times, where n = MaxInst
df_pred_li = df_pred.reindex(df_pred.index.repeat(df_pred['MaxInst'])).reset_index(drop=True)
# add installment number
df_pred_li.loc[:, 'InstallmentNumber'] = df_pred_li.groupby('LoanID').cumcount() + 1

In [57]:
# seperate out installments that has been paid (no need to predict the outcome)
mask = df_pred_li['InstallmentNumber'] <= df_pred_li['MaxInstPaidCont'].fillna(0)
df_paid_li = df_pred_li[mask].copy()
df_pred_li = df_pred_li[~mask]

#### Introduce the Predictors into the Loan-Installment Dataset

In [58]:
# reversed installment number
df_pred_li['ReversedInst'] = df_pred_li['MaxInst'] - df_pred_li['InstallmentNumber']

# create interactions between Installment and DefScore/EPOScore
interactions = ['InstDefScore', 'InstEPOScore']
df_pred_li.loc[:, 'InstDefScore'] = df_pred_li.loc[:, 'InstallmentNumber'] * df_pred_li.loc[:, 'DefScore']
df_pred_li.loc[:, 'InstEPOScore'] = df_pred_li.loc[:, 'InstallmentNumber'] * df_pred_li.loc[:, 'EPOScore']

#### Apply multinomial logistic models to df_pred_li 

In [59]:
# create an empty list to store prediction results for different pay frequency
l_res = list()

In [60]:
def make_prediction(data, freq, d_lr, d_var):
    # build model for a specific product
    df_li = data[data['Frequency'] == freq].copy()
    
    # one-hot encoding
    # add a set of dummy predictors to represent installment generically
    df_time_ind = pd.get_dummies(df_li['InstallmentNumber'], prefix='D', prefix_sep='')
    df_li = pd.concat([df_li, df_time_ind], axis = 1)
    
    # add a set of dummy predictors to represent age groups
    df_age_group = pd.get_dummies(df_li['AgeGroup'], prefix='Age', prefix_sep='_')
    df_li = pd.concat([df_li, df_age_group], axis = 1)
    
    # in case some variables are not presented in test set
    for var in d_var[freq]:
        if var not in df_li.columns:
            df_li[var] = 0
    
    # apply the multinomial logistic regression to data
    X = df_li[d_var[freq]] 
    df_res = d_lr[freq].predict(X)
    df_res.columns = ['PCHazard', 'DefHazard', 'EPOHazard']
    
    # append LoanID and InstallmentNumber to prediced results
    df_res = pd.concat([df_li[['LoanID', 'InstallmentNumber']], df_res], axis=1)
    
    return df_res

In [61]:
mask = df_pred_li['DefScore'].notnull() & df_pred_li['EPOScore'].notnull()

for freq in ['B', 'W', 'M']:
    l_res.append(make_prediction(df_pred_li[mask], freq, d_lr, d_var))
    l_res.append(make_prediction(df_pred_li[~mask], freq, d_lr_wos, d_var_wos))
    
# concatenate the results into one dataframe
df_res = pd.concat(l_res)
df_res.head(2)

,LoanID,InstallmentNumber,PCHazard,DefHazard,EPOHazard
0,I2117479-0,1,0.766004,0.077775,0.156221
1,I2117479-0,2,0.813132,0.091513,0.095355


In [62]:
# append predictions to installment-level dataset
df_pred_li = df_pred_li.merge(df_res, how='left', on=['LoanID', 'InstallmentNumber'])

#### Use non-parametric estimation to fill the missing PCHazard/DefHazard/EPOHazard

In [63]:
df_pred_li = df_pred_li.merge(np_survival[['Frequency', 'InstallmentNumber', 'PCHazard', 'DefHazard', 'EPOHazard']], 
                              how='left', on=['Frequency', 'InstallmentNumber'], 
                              suffixes=['', '_np'])

for hazard in ['PCHazard', 'DefHazard', 'EPOHazard']:
    df_pred_li.loc[:, hazard] = df_pred_li[hazard].fillna(df_pred_li[f'{hazard}_np'])

    
# EPOHazard and DefHazard must add up to 1 for the last installment
def adjust_epo_harzard(df):
    df.iloc[-1, df.columns.get_loc('EPOHazard')] = 1 - df['DefHazard'].iloc[-1]
    df.iloc[-1, df.columns.get_loc('PCHazard')] = 0
    return df

df_pred_li = df_pred_li.groupby('LoanID').apply(adjust_epo_harzard)

# calculate survival probability
df_pred_li['SurvivalProb'] = df_pred_li.groupby('LoanID')['PCHazard'].apply(pd.Series.cumprod)
df_pred_li['SurvivalProbPreInst'] = df_pred_li.groupby('LoanID')['SurvivalProb'].apply(lambda s: s.shift(1).fillna(1))

# calculate absolute risk
df_pred_li['DefAbsRisk'] = df_pred_li['DefHazard'] * df_pred_li['SurvivalProbPreInst']
df_pred_li['EPOAbsRisk'] = df_pred_li['EPOHazard'] * df_pred_li['SurvivalProbPreInst']

# calculate cumulative default/early-payoff rate
df_pred_li['CumDefRate'] = df_pred_li.groupby('LoanID')['DefAbsRisk'].apply(pd.Series.cumsum)
df_pred_li['CumEPORate'] = df_pred_li.groupby('LoanID')['EPOAbsRisk'].apply(pd.Series.cumsum)

In [64]:
df_paid_li['DefAbsRisk'] = 0
df_paid_li['EPOAbsRisk'] = 0

df_pred_li = pd.concat([df_paid_li, df_pred_li], sort=False)\
.sort_values(['LoanID', 'InstallmentNumber'])

#### Calculate Expected Payin

In [65]:
# get calibration factors by frequency by installment
df_pred_li = df_pred_li.merge(df_cf[['Frequency', 'InstallmentNumber', 'RR', 'CFD', 'CFE']], 
                              how='left')

In [66]:
# get original repayment schedule for each loan
df_pred_li = df_pred_li.merge(df_inst, on=['LoanID', 'InstallmentNumber'], how='left')

In [67]:
# calculate conditional payin by installment for a given loan
def get_cond_payin_by_inst(data):
    # pre-period principal balance
    data['OriginalPrincipalBalance'] \
    = data['OriginatedAmount'].iloc[0] - data['OriginalPrincipal'].shift(1).fillna(0).cumsum()
    # theoritical payin if default occurs at installment i (assuming recovery rate = 0)
    data['TPDByInst'] = data['OriginalDueAmount'].shift(1).fillna(0).cumsum()
    # theoritical payin if customer completely pay off after installment i
    data['TPEByInst'] = data['OriginalDueAmount'].cumsum() - data['OriginalPrincipal'] \
    + data['OriginalPrincipalBalance']
    # calibrated payin if default occurs at installment i
    # pre-default
    data['CPDPreByInst'] = data['CFD'] * data['TPDByInst']
    # post-default
    data['CPDPostByInst'] = data['RR'] * data['OriginalPrincipalBalance']
    # calibrated payin if customer completely pay off after installment i
    data['CPEByInst'] = data['CFE'] * data['TPEByInst']
    return data

In [68]:
# get conditional payin by installment for each loan
df_pred_li = df_pred_li.groupby('LoanID').apply(get_cond_payin_by_inst)\
.reset_index(level=0, drop=True)

In [69]:
# calculate expected payin for a given loan
def get_exp_payin(data):
    # drop installments that have been paid
    data = data[data['PCHazard'].notnull()]
    exp_payin_pre_default = (data['DefAbsRisk'] * data['CPDPreByInst'] + data['EPOAbsRisk'] * data['CPEByInst']).sum()
    exp_payin_post_default = (data['DefAbsRisk'] * data['CPDPostByInst']).sum()
    exp_payin = exp_payin_pre_default + exp_payin_post_default
    return pd.Series({'ExpPayinPreDefault': exp_payin_pre_default, 
                      'ExpPayinPostDefault': exp_payin_post_default,
                      'ExpectedPayin': exp_payin})

In [70]:
# get expected payin for each loan
df_exp_payin = df_pred_li.groupby('LoanID').apply(get_exp_payin)

In [71]:
# append expected payin to df
df = df.merge(df_exp_payin, how='left', left_on='LoanID', right_index=True)

### For defauted loans  

In [72]:
# append calibration factors to defaulted loans
df = df.merge(df_cf[['Frequency', 'InstallmentNumber', 'RR', 'CFD']], how='left', 
              left_on=['Frequency', 'InstDefaulted'], 
              right_on=['Frequency', 'InstallmentNumber'])

In [73]:
# get expected payin of defaulted loans
mask = df['MyLoanStatus'] == 'defaulted'
df.loc[mask, 'ExpPayinPreDefault'] = df.loc[mask, 'InstPayinPreDefault']
df.loc[mask, 'ExpPayinPostDefault'] = df.loc[mask, 'RR'] * df.loc[mask, 'PrincipalBalance']
df.loc[mask, 'ExpectedPayin'] = df.loc[mask, 'ExpPayinPreDefault'] + df.loc[mask, 'ExpPayinPostDefault']

errored accounts

In [74]:
# append calibration factors to defaulted loans
df_error = df_error.merge(df_cf[['Frequency', 'InstallmentNumber', 'RR', 'CFD']], how='left', 
                          left_on=['Frequency', 'InstDefaulted'], 
                          right_on=['Frequency', 'InstallmentNumber'])

In [75]:
# get expected payin of defaulted loans
mask = df_error['MyLoanStatus'] == 'defaulted'
df_error.loc[mask, 'ExpPayinPreDefault'] = df_error.loc[mask, 'InstPayinPreDefault']
df_error.loc[mask, 'ExpPayinPostDefault'] = df_error.loc[mask, 'RR'] * df_error.loc[mask, 'PrincipalBalance']
df_error.loc[mask, 'ExpectedPayin'] = df_error.loc[mask, 'ExpPayinPreDefault'] + df_error.loc[mask, 'ExpPayinPostDefault']

### For inactive loans 

In [76]:
# loans that have been paid off
mask = df['InstPaidOff'].notnull()
df.loc[mask, 'ExpPayinPreDefault'] = df.loc[mask, 'PayinToDate']
df.loc[mask, 'ExpectedPayin'] = df.loc[mask, 'PayinToDate']
df.loc[mask, 'ExpPayinPostDefault'] = 0

In [77]:
# other inactive loans
mask = (~df['MyLoanStatus'].isin(['current', 'defaulted'])) & df['InstPaidOff'].isnull()
df.loc[mask, 'ExpPayinPreDefault'] = df.loc[mask, 'InstPayinPreDefault'].fillna(0)
df.loc[mask, 'ExpectedPayin'] = df.loc[mask, 'PayinToDate']
df.loc[mask, 'ExpPayinPostDefault'] = df.loc[mask, 'ExpectedPayin'] - df.loc[mask, 'ExpPayinPreDefault']

In [78]:
df.loc[df['ExpectedPayin'].isnull(), 'Error'] = 'unknown'

errored accounts

In [79]:
mask = df_error['InstPaidOff'].notnull()
df_error.loc[mask, 'ExpPayinPreDefault'] = df_error.loc[mask, 'PayinToDate']
df_error.loc[mask, 'ExpectedPayin'] = df_error.loc[mask, 'PayinToDate']
df_error.loc[mask, 'ExpPayinPostDefault'] = 0

In [80]:
mask = (~df_error['MyLoanStatus'].isin(['current', 'defaulted'])) & df_error['InstPaidOff'].isnull()
df_error.loc[mask, 'ExpPayinPreDefault'] = df_error.loc[mask, 'InstPayinPreDefault'].fillna(0)
df_error.loc[mask, 'ExpectedPayin'] = df_error.loc[mask, 'PayinToDate']
df_error.loc[mask, 'ExpPayinPostDefault'] = df_error.loc[mask, 'ExpectedPayin'] - df_error.loc[mask, 'ExpPayinPreDefault']

# Get Final Results

In [81]:
df_final = pd.concat([df, df_error], sort=False).sort_values(by='LoanID')

In [82]:
# number of loans without ExpectedPayin by error types
df_final.loc[df_final['ExpectedPayin'].isnull(), 'Error'].value_counts()

unknown    5
Name: Error, dtype: int64

In [83]:
# default/early-payoff probabilities by installment
# get the absolute risk of accounts that have defaulted or paid off
df_realized = df[df[['InstDefaulted', 'InstPaidOff']].notnull().any(axis=1)]
df_realized_li = df_realized[['LoanID', 'InstDefaulted', 'InstPaidOff']]\
.merge(df_inst, how='left', on='LoanID')

mask = df_realized_li['InstallmentNumber'] == df_realized_li['InstDefaulted']
df_realized_li['DefAbsRisk'] = np.where(mask, 1, 0)

mask = df_realized_li['InstallmentNumber'] == df_realized_li['InstPaidOff']
df_realized_li['EPOAbsRisk'] = np.where(mask, 1, 0)

# whether the absolute risk is predicted or not
df_realized_li['Predicted'] = 0
df_pred_li['Predicted'] = np.where(df_pred_li['PCHazard'].isnull(), 0, 1)

cols_to_display = ['LoanID', 'InstallmentNumber', 'DefAbsRisk', 'EPOAbsRisk', 'Predicted']
df_realized_li = df_realized_li[cols_to_display]

# combine the absolute risk of all accounts
df_abs_risk = pd.concat([df_realized_li, df_pred_li[cols_to_display]])
df_abs_risk['UpdatedDate'] = df_final['UpdatedDate'].iloc[0]

# Export Results

### For backtesting 

In [84]:
# export loan-level results
# df_final.to_pickle('../Data/For_Backtest/latest_perf.pkl')
# export installment-level results
# df_abs_risk.to_pickle('../Data/For_Backtest/latest_perf_inst.pkl')

### Write to DB

In [85]:
final_cols = ['LoanID', 'OriginationDate', 'OriginatedAmount', 'LoanStatus', 'Frequency', 'Benefits', 
              'DefScore', 'EPOScore', 'Age', 'MaxInst', 'InstDefaulted', 'InstPaidOff', 'MaxInstPaidCont', 
              'PayinToDate', 'PrincipalBalance', 'InstPayinPreDefault', 'InstPayinPostDefault', 'ArrPayin',
              'DateOfDefault', 'DateOfLastPayment', 'UpdatedDate', 'MyLoanStatus', 'MostCommIntRate', 'Error', 
              'Warning', 'AgeGroup', 'EventTime', 'EventType', 'RR', 'CFD', 
              'ExpPayinPreDefault', 'ExpPayinPostDefault', 'ExpectedPayin']

In [86]:
# df_final[final_cols].to_csv('PayinProj_Output.csv')

In [87]:
# connect to DB
server = '192.168.1.15'
username = 'CChang'
password = 'summerjob2017'
params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE=Qlikdb;UID='+username+';PWD='+ password)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#  loan-level outputs
df_final[final_cols].to_sql(name='PayInPredictions_New', schema='dbo', con=engine, if_exists='append', index=False)

6

In [88]:
len(df_final)

79890

In [89]:
# df_final[final_cols].to_csv('sample.csv')
cnxn.close()
engine.dispose()

In [90]:
# installment-level outputs
# df_abs_risk.to_sql(name='DefEPOProb', schema='dbo', con=engine, if_exists='append', index=False)

old payin projection table

In [91]:
# modify the column names to fit into the PayinPredictions table in DB
# df_final['Active'] = df_final['MyLoanStatus'].isin(['current', 'defaulted']).astype(int)
# df_final['MaxInstNoDue'] = df_final['EventTime']
# df_final['FirstInstallmentDue'] = (df_final['MaxInstNoDue'] >= 1).astype(int)

# df_to_db = df_final[['LoanID', 'Frequency', 'OriginationDate', 'OriginatedAmount', 'Active', 
#                      'UpdatedDate', 'FirstInstallmentDue', 'MaxInstPaidCont', 'InstDefaulted', 
#                      'InstPayinPreDefault', 'InstPayinPostDefault', 'ArrPayin', 
#                      'PayinToDate', 'ExpectedPayin', 'LoanStatus', 'MaxInstNoDue', 
#                      'InstPaidOff']].copy()
# df_to_db.columns = ['LoanID', 'Frequency', 'OriginationDate', 'OriginatedAmount', 'Active', 
#                     'UpdatedDate', 'FirstInstallmentDue', 'MaxInstNoPaidContinue', 
#                     'InstNoDefaultout', 'RealizedInstPayPreDefault', 'RealizedInstPayPostDefault',
#                     'RealizedArrPay', 'RealizedTotalPay', 'expected_tot_payin', 'LoanStatus', 
#                     'MaxInstNoDue', 'InstNoPaidOff']

In [92]:
# connect to DB
# server = '192.168.1.15'
# username = 'CChang'
# password = 'summerjob2017'
# params = urllib.parse.quote_plus('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE=Qlikdb;UID='+username+';PWD='+ password)
# engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
# df_to_db.to_sql(name='PayInPredictions', schema='dbo', con=engine, if_exists='append', 
#                 index=False)

In [94]:
# logging.exception('AccountID: ' + str(bankaccountid) + '; NDBAccount: ' + str(ndbaccountid) + '; ErrorInStep: ' + stepname + ';RF Input:' + str(clf_ibv.feature_names_in_) + str(len(clf_ibv.feature_names_in_)))